>[Project aims](#Project-aims)

>[Project aims](#project-aims)

>[Physical repairs](#scrollTo=DrzTjSct7gwI)

>[Electronics](#scrollTo=9_P7qOVd8PtY)

>[Angle limits](#scrollTo=fhELyI_DDX8T)

>>[Diagrams](#scrollTo=fhELyI_DDX8T)

>[Basic angles](#scrollTo=LwVAr5ma4aKC)

>>[a: base (yaw)](#scrollTo=LwVAr5ma4aKC)

>>[b: shoulder (pitch)](#scrollTo=LwVAr5ma4aKC)

>>[c: elbow (pitch)](#scrollTo=LwVAr5ma4aKC)

>>[d: wrist (pitch)](#scrollTo=LwVAr5ma4aKC)

>>[e: gripper jaws](#scrollTo=LwVAr5ma4aKC)



TODO:
fix diagram to show external angles not internal ones

# Project aims
* Repair parts as needed so all parts work
* Find a better way to power the robot
* Communicate with the controller board over USB from python (motors on/off/reverse etc)
* Create a program to:
    * Work out the location of the end of the arm from the angles of the joints
    * find a solution of angles to get the end of the arm to a given position in 3D space
    * (approximately) move a motor by a specified angle, from the motor having a known rotational speed
    * not allow invalid movements (eg below ground plane)
* Find a better way to track the motors' current rotation position


# Physical repairs
* Motor B's connection to the base had been pulled apart, and the motor/gearbox part required reassembly
* Motor B's wires had come off, so I resoldered them back on
* Multiple motors had not moved for a long time, and were not able to move. I fixed this by taking apart the gear box for each motor and allowing the motor to spin freely for a few seconds
* The batteries were flat (but I had no D Cells availiable), so I used 2 AA batteries instead

# Electronics


*Had to work out how to power the robot, if possible without needing to buy D cells
* had to work out circuitry -- The board seemed to use two different voltages from the battery compartment
* Reverse engineering circuit
* Looking up ICs
  * USB Controller
  * Motor Controllers (2 types)

# Diagrams


![Robot arm](Photos/IMG_20200331_180758%20-%20IMG_20200331_180804_1.jpg)
![Robot arm diagram](Photos/IMG_20200331_180758%20-%20IMG_20200331_180804_2_small.jpg)

|Point|Name|Axis|
|--|--|--|
|a|Base|yaw|
|b|Shoulder|pitch|
|c|Elbow|pitch|
|d|Wrist|pitch|
|e|Gripper jaws||

|Line|Length (mm)|
|----|------|
|$ab$|65    |
|$bc$|90    |
|$cd$|62    |
|$de$|90-115|

(point e is at the end of the jaws)

# Kinematics

<img src="https://upload.wikimedia.org/wikipedia/commons/f/f4/FWDvsINV_Kinematics_HighResTransp.png" alt="FWDvsINV Kinematics HighResTransp.png" style="max-height: 16em;margin: 0;">
<!--<img src="https://upload.wikimedia.org/wikipedia/commons/f/f4/FWDvsINV_Kinematics_HighResTransp.png" alt="FWDvsINV Kinematics HighResTransp.png">-->
Image By <a href="//commons.wikimedia.org/w/index.php?title=User:Haendy-freak&amp;action=edit&amp;redlink=1" class="new" title="User:Haendy-freak (page does not exist)">Haendy-freak</a> - <span class="int-own-work" lang="en">Own work</span>, <a href="https://creativecommons.org/licenses/by-sa/4.0" title="Creative Commons Attribution-Share Alike 4.0">CC BY-SA 4.0</a>, <a href="https://commons.wikimedia.org/w/index.php?curid=86278722">Link</a>

## Forward kinematics

Forward kinematics involves calculating the position of the robot's jaws from current angles of the joints, and the lengths of the segments. We can do this using trigonometry:

In [3]:
from math import sin, cos, degrees, radians
class Coord2D:
  def __init__(self, x,y):
    self.x = x
    self.y = y
  @classmethod
  def from_polar(cls,r,theta):
    return cls(r*cos(theta),r*sin(theta))
  @classmethod
  def from_polar_degrees(cls,r,theta):
    return cls(r*cos(radians(theta)),r*sin(radians(theta)))
  def __add__(self, other):
    return self.__class__(self.x+other.x,self.y+other.y)
  def __repr__(self):
    return f'({self.x},{self.y})'

line_lengths = {
  'ab': 6.5e-2,
  'bc': 9.0e-2,
  'cd': 6.2e-2,
  'de': 1.15e-1
}
L = line_lengths
a = Coord2D(0,0)
b = a + Coord2D(0,line_lengths['ab'])
c = b + Coord2D.from_polar_degrees(line_lengths['bc'],90)
d = c + Coord2D.from_polar_degrees(line_lengths['cd'],90)
e = d + Coord2D.from_polar_degrees(line_lengths['de'],0)
print(e)

e = Coord2D(0,0) + Coord2D(0,L['ab']) + Coord2D.from_polar_degrees(L['bc'],90) + Coord2D.from_polar_degrees(L['cd'],90) + Coord2D.from_polar_degrees(L['de'],0)
print(e)

(0.11500000000000002,0.217)
(0.11500000000000002,0.217)


## Inverse kinematics
Inverse kinematics involves calculating the angles of the different joints for the end of the arm to be at a given position.

This process is much more complicated than forward kinematics in complex robots, however the robot arm i am using is relatively simple; there are 3 pitch axes, and one yaw axis (a). This means that for a point in space that the robot can reach, there is only one value for the yaw axis. This leaves us with a 2D problem of the angles for the pitch axes (angles at b, c and d)

The vector for the final point is this:

![Robot arm diagram](Photos/IMG_20200331_180758%20-%20IMG_20200331_180804_2_smaller.jpg)

$\begin{pmatrix}0\\0\end{pmatrix}+
\begin{pmatrix}0\\6.5{\times}10^{-2}\end{pmatrix}+
\begin{pmatrix}9.0{\times}10^{-2} \times cos(b)\\9.0{\times}10^{-2} \times sin(b)\end{pmatrix}+
\begin{pmatrix}6.2{\times}10^{-2} \times cos(c)\\6.2{\times}10^{-2} \times sin(c)\end{pmatrix}+
\begin{pmatrix}1.15{\times}10^{-1} \times cos(d)\\1.15{\times}10^{-1} \times sin(d)\end{pmatrix}$

$\begin{pmatrix}
x\\y
\end{pmatrix}=
\begin{pmatrix}
0+(9.0{\times}10^{-2} \times cos(b))+(6.2{\times}10^{-2} \times cos(c))+(1.15{\times}10^{-1} \times cos(d)\\
(6.5{\times}10^{-2})+(9.0{\times}10^{-2} \times sin(b))+(6.2{\times}10^{-2} \times sin(c))+(1.15{\times}10^{-1} \times sin(d)\end{pmatrix}$


$
\begin{align*}
0^\circ < \angle b &< 90^\circ \\
-135^\circ < \angle c &< 135^\circ \\
-45^\circ < \angle d &< 45^\circ \\
\end{align*}
$

$\begin{align}
x &= 0+(9.0{\times}10^{-2} \times cos(b))+(6.2{\times}10^{-2} \times cos(c))+(1.15{\times}10^{-1} \times cos(d)\\
y &= (6.5{\times}10^{-2})+(9.0{\times}10^{-2} \times sin(b))+(6.2{\times}10^{-2} \times sin(c))+(1.15{\times}10^{-1} \times sin(d)
\end{align}$


$
\begin{align*}
-90^\circ < \angle a &< 90^\circ \\
90^\circ < \angle b &< 180^\circ \\
45^\circ < \angle c &< 315^\circ \\
135^\circ < \angle d &< 225^\circ \\
0^\circ < \angle e &< 90^\circ
\end{align*}
$

## Angle constraints
$
\begin{align*}
-90^\circ < \angle a &< 90^\circ \\
90^\circ < \angle b &< 180^\circ \\
45^\circ < \angle c &< 315^\circ \\
135^\circ < \angle d &< 225^\circ \\
0^\circ < \angle e &< 90^\circ
\end{align*}
$

In general, large angles should be avoided, because this can cause collisions between parts of the arm

# Collision detection
To work out limits we create a "unsafe volume" around each line.
TODO: do this because some positions cannot be reached eg. on top of base (runtime sanity check?)

# Links/further reading
## Python
* https://google.github.io/styleguide/pyguide.html
## Robotic arm
* https://appliedgo.net/roboticarm/
* http://mattdyson.org/projects/robotarm/
* https://notbrainsurgery.livejournal.com/38622.html
## Kinematics
* https://en.wikipedia.org/wiki/Forward_kinematics
* https://en.wikipedia.org/wiki/Inverse_kinematics
* https://en.wikipedia.org/wiki/Kinematic_chain
* https://en.wikipedia.org/wiki/Degrees_of_freedom_(mechanics)
* https://en.wikipedia.org/wiki/321_kinematic_structure
    * https://apps.dtic.mil/dtic/tr/fulltext/u2/680036.pdf
* https://en.wikipedia.org/wiki/Arm_solution
* https://en.wikipedia.org/wiki/Cylindrical_coordinate_system
## Other
* https://en.wikipedia.org/wiki/Pulse-width_modulation

In [9]:
import numpy
import math
vec = (1,1,1)
print(type(numpy.linalg.norm(vec)))
print(type(math.sqrt((vec[0]**2)+(vec[1]**2)+(vec[2]**2))))

type(vec/numpy.linalg.norm(vec))

<class 'numpy.float64'>
<class 'float'>


numpy.ndarray